In [ ]:
import spacy

import pandas as pd
import os
from tqdm import tqdm
from spacy.tokens import DocBin

In [ ]:
ner = spacy.load("en_core_web_sm")

In [ ]:
import json
def create_dataset(path):
    datasets = []
    with open(path, 'r') as f:
        datas = json.load(f)

    for data in datas:
        entities = []
        for entity in data['annotations'][1]['entities']:
            entities.append((entity[0], entity[1], entity[2]))
        datasets.append((data['annotations'][0], {"entities": entities}))

    return datasets

path = "/content/dex_annotations.json"

train = create_dataset(path)


In [ ]:
db = DocBin() # create a DocBin object

for text, annot in tqdm(train): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy")

In [ ]:
!!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

In [ ]:
ner1 = spacy.load(r"./output/model-best")